<a href="https://colab.research.google.com/github/sszobaer/RAG-Model-Simulation/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **RAG SIMULATION**

In [3]:
!pip install torch transformers datasets faiss-cpu tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### **Load Corpus**

In [10]:
from datasets import Dataset

# Example corpus
documents = [
    "What is machine learning?",
    "Machine learning is a subfield of artificial intelligence.",
    "It involves training models on data to make predictions.",
    "And we are using this as our research domain."
]

# Convert the corpus into a Hugging Face Dataset
dataset = Dataset.from_dict({"text": documents})

# Display the dataset
dataset

Dataset({
    features: ['text'],
    num_rows: 4
})

### **Create a FAISS Index for Retrieval**

In [11]:
from transformers import AutoTokenizer, AutoModel
import faiss
import torch

# Load a transformer model for embeddings (using Sentence-BERT model)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to embed documents
def embed_documents(texts):
    """Embed the texts using the transformer model."""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Embed the corpus and build the FAISS index
embeddings = embed_documents(dataset["text"])
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance for retrieval
index.add(embeddings)  # Add embeddings to the index

# Check the index information
print('The total index number is: ', index.ntotal)

The total index number is:  4


### **Perform Retrieval**

In [18]:
def retrieve(query, top_k=3):
    """Retrieve top_k documents for a given query."""
    query_embedding = embed_documents([query])
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the documents based on the indices
    results = [dataset[i.item()]["text"] for i in indices[0]]
    return results

# Test retrieval
query = "What is models?"
retrieved_docs = retrieve(query)
retrieved_docs

['It involves training models on data to make predictions.',
 'And we are using this as our research domain.',
 'Machine learning is a subfield of artificial intelligence.']

### **Generate an Answer Using a Language Model**

In [27]:
from transformers import pipeline

# Load a text generation pipeline with GPT-2
qa_pipeline = pipeline("text-generation", model="gpt2")

# Generate a response based on the retrieved documents
context = " ".join(retrieved_docs)
response = qa_pipeline(context, max_length=50, num_return_sequences=1)

response[0]["generated_text"]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"It involves training models on data to make predictions. And we are using this as our research domain. Machine learning is a subfield of artificial intelligence.\n\nWe've also been involved in the recent research in a variety of different domains of human intelligence"

 ### **Enable GPU in Google Colab**

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [25]:
inputs = tokenizer(query, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [26]:
qa_pipeline = pipeline("text-generation", model="gpt2", pad_token_id=50256)

Device set to use cpu
